In [30]:
import numpy as np
import csv
import sys
sys.path.append("..")
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from Code.VQLSSVM import VQLSSVM
from Code.Utils import prepareDataset

np.set_printoptions(precision=10, suppress=True)

In [31]:
qubits = 4
shots: int = 10000
gamma: float = 0.01 # regularization parameter
subsetSize: int = 2**qubits - 1 # number of training points
classToFilterOut: int = 2
iterations = 1
trainIterations = 200
datasets = ["iris"]
# datasets = ["iris","breastCancer"]

In [32]:
vqlssvmVectors: VQLSSVM = VQLSSVM(gamma, shots)

In [33]:
def getListsAverage(data: list):
    maximumLengthList = len(max(data, key=len))
    sumList =[]
    for i in range(maximumLengthList):
        iterationSum = 0
        for j in range(len(data)):
            if i < len(data[j]):
                iterationSum += data[j][i]
            else:
                iterationSum += data[j][-1]
        sumList.append(iterationSum)

    sumList = np.array(sumList)
    return sumList/len(data)

In [34]:
def collectTrainData(dataset: str, isQuantumSimulation: bool):
    xTrain, xTest, yTrain, yTest = prepareDataset(normalizeValues=True, dataset=dataset,subsetSize=subsetSize, classToFilterOut=classToFilterOut)
    vqlssvmVectors.train(xTrain, yTrain, quantumSimulation=isQuantumSimulation, verbose=False, iterations = trainIterations, method="COBYLA")
    cost = vqlssvmVectors.getCostHistory()
    accuracy = vqlssvmVectors.accuracy(xTest, yTest)
    clf = SVC(kernel='linear')
    clf.fit(xTrain, yTrain)
    yPred = clf.predict(xTest)
    return cost, accuracy, accuracy_score(yTest, yPred)

costs = {}
accuracies = {}
accuraciesSVM = {}

for dataset in datasets:
    costs[dataset] = []
    accuracies[dataset] = []
    accuraciesSVM[dataset] = []

for i in range(iterations):
    print(i,"th iteration")
    for dataset in datasets:
        print("Dataset:",dataset)
        cost,accuracy,svmAccuracy = collectTrainData(dataset, True)
        costs[dataset].append(cost)
        accuracies[dataset].append(accuracy)
        accuraciesSVM[dataset].append(svmAccuracy)
        print("\n")
    print("\n")

0 th iteration
Dataset: iris


capi_return is NULL
Call-back cb_calcfc_in__cobyla__user__routines failed.
Traceback (most recent call last):
  File "/Users/pijus/.pyenv/versions/3.11.4/lib/python3.11/site-packages/scipy/optimize/_cobyla_py.py", line 281, in calcfc
Fatal Python error: F2PySwapThreadLocalCallbackPtr: F2PySwapThreadLocalCallbackPtr: PyLong_AsVoidPtr failed
Python runtime state: initialized
    f = sf.fun(x)
        ^^^^^^^^^
  File "/Users/pijus/.pyenv/versions/3.11.4/lib/python3.11/site-packages/scipy/optimize/_differentiable_functions.py", line 267, in fun
    self._update_fun()
  File "/Users/pijus/.pyenv/versions/3.11.4/lib/python3.11/site-packages/scipy/optimize/_differentiable_functions.py", line 251, in _update_fun
    self._update_fun_impl()
  File "/Users/pijus/.pyenv/versions/3.11.4/lib/python3.11/site-packages/scipy/optimize/_differentiable_functions.py", line 155, in update_fun
    self.f = fun_wrapped(self.x)
             ^^^^^^^^^^^^^^^^^^^
  File "/Users/pijus/.pyenv/versions/3.11.4/lib/

: 

: 

In [ ]:
for dataset in datasets:
    with open('../SimulationResults/costDataset' + dataset+'.csv', 'w', newline='') as csvfile:
        avgCosts = {}
        length = costs[dataset].__len__()
        s = np.array([sum(a) for a in zip(*costs[dataset])])
        avgCosts[dataset] = getListsAverage(costs[dataset])
        
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(['Iteration'] + [dataset])
        combined = [ [i] + [x] for i, x in enumerate(avgCosts[dataset])]
        writer.writerows(combined)

with open('../SimulationResults/accuracyDataset.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    titleRow = []
    dataRow = []
    for dataset in datasets:
        titleRow.append(dataset)
        titleRow.append(dataset + "SVM")
        dataRow.append(np.mean(accuracies[dataset]))
        dataRow.append(np.mean(accuraciesSVM[dataset]))
    writer.writerow(titleRow)
    writer.writerow(dataRow)